In [1]:
import numpy as np
import pandas as pd
import pydicom
from keras.saving.model_config import model_from_json
%matplotlib inline

In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print(f'Load file {filename} ...')
    ds = pydicom.dcmread(filename)
    img = ds.pixel_array
    img_mean = img.mean()
    img_std = img.std()
    img_type = ds.Modality
    img_position = ds.PatientPosition
    img_body_part = ds.BodyPartExamined
    patient_age = ds.PatientAge
    min_age = 1
    max_age = 90

    if img_type != 'DX':
        print(f'Image type not DX, but {img_type}')
        return None, None, None
    if img_position not in ['AP', 'PA']:
        print(f'Patient position not AP or PA, but {img_position}')
        return None, None, None
    if img_body_part != 'CHEST':
        print(f'Body part not CHEST, but {img_body_part}')
        return None, None, None
    if not min_age <= int(patient_age) <= max_age:
        print(f'Patient age not in range {min_age}-{max_age}, but {patient_age}')
        return None, None, None
    

    # Print relevant fields
    print(f"Image position: {ds.PatientPosition}")
    print(f"Image type: {ds.Modality}")
    print(f"Body part examined: {ds.BodyPartExamined}")
    print(f"Study description: {ds.StudyDescription}")
    print(f"image mean: {img_mean}")
    print(f"image std: {img_std}")

    return ds.pixel_array, img_mean, img_std
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size):
    proc_img = img
    proc_img = (proc_img - img_mean) / img_std
    proc_img = np.resize(proc_img, img_size)

    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, 'r') as json_file:
        model = model_from_json(json_file.read())
    model.load_weights(weight_path)

    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh):
    prediction = model.predict(img)

    return prediction > thresh

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path = "xray_class_my_model.best.weights.h5"

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 0 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.6 #loads the threshold they chose for model classification

# use the .dcm files to test your prediction
for i in test_dicoms:
    img, img_mean, img_std = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img, img_mean, img_std, IMG_SIZE)
    print(predict_image(my_model, img_proc, thresh))

Load file test1.dcm ...
Image position: PA
Image type: DX
Body part examined: CHEST
Study description: No Finding
image mean: 180.0586576461792
image std: 47.89115072730743
1/1 [==============================] - 1s 947ms/step
[[False]]
Load file test2.dcm ...
Image position: AP
Image type: DX
Body part examined: CHEST
Study description: Cardiomegaly
image mean: 128.13570880889893
image std: 67.42323200014457
1/1 [==============================] - 0s 19ms/step
[[ True]]
Load file test3.dcm ...
Image position: AP
Image type: DX
Body part examined: CHEST
Study description: Effusion
image mean: 104.74800395965576
image std: 66.22413614301003
1/1 [==============================] - 0s 18ms/step
[[ True]]
Load file test4.dcm ...
Body part not CHEST, but RIBCAGE
Load file test5.dcm ...
Image type not DX, but CT
Load file test6.dcm ...
Patient position not AP or PA, but XX
